In [ ]:
import matplotlib.pyplot as plt
import deepxde as dde
import numpy as np
import torch

from modules.data import disc_to_cont, data_obj as data

device = "cuda"

In [ ]:
geom = dde.geometry.TimeDomain(data.t[0], data.t[-1])

M = torch.Tensor(data.M).to(device)
C = torch.Tensor(data.M).to(device)
K = torch.Tensor(data.M).to(device)

def PDE(x, y, v):
    """Defines the governing PDE.

    Args:
        x (torch.Tensor): input function evaluation points
        v (torch.Tensor): input function
        y (torch.Tensor): output function
    """

    N = y.shape[1]

    u1 = 
    u3 = y[:, :N//2]
    u4 = y[:, N//2:]
